# 🔥 VTrackAI Studio - Kaggle Stable (No Tunnels)

**Prerequisites:**
1. Settings → Accelerator → **GPU P100** or **T4 x2**
2. Add-ons → Secrets → Add `HF_TOKEN` (from huggingface.co/settings/tokens)
3. Enable **Internet** in the sidebar

**No tunnel passwords needed!** Uses Kaggle's native External Link feature.

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 1: SETUP (Run once per session, ~3-5 min)
# ═══════════════════════════════════════════════════════════════════════════════
import torch
import os
import sys
import subprocess

BASE_DIR = "/kaggle/working"
os.chdir(BASE_DIR)

# GPU Check
print("="*60)
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ NO GPU! Go to Settings → Accelerator → GPU")
    sys.exit(1)
print("="*60)

# Clean start
subprocess.run(["rm", "-rf", f"{BASE_DIR}/vtrack-ai-studio"])

# Install dependencies
print("\n📦 Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "torch", "torchvision", "--index-url", "https://download.pytorch.org/whl/cu121"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "fastapi", "uvicorn[standard]", "python-multipart", "huggingface_hub", "requests", "decord"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "numpy==1.26.4"])

# Node.js 18 via nodeenv
print("\n📦 Installing Node.js 18...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "nodeenv"])
subprocess.run(["nodeenv", "--node=18.20.0", f"{BASE_DIR}/node_env", "--force", "-q"])

NODE_BIN = f"{BASE_DIR}/node_env/bin"
os.environ["PATH"] = f"{NODE_BIN}:{os.environ['PATH']}"

subprocess.run([f"{NODE_BIN}/node", "--version"])

# Clone VTrackAI
print("\n📦 Cloning VTrackAI...")
subprocess.run(["git", "clone", "-q", "https://github.com/Amitro123/vtrack-ai-studio", f"{BASE_DIR}/vtrack-ai-studio"])
subprocess.run([sys.executable, "-m", "pip", "install", "-r", f"{BASE_DIR}/vtrack-ai-studio/backend/requirements.txt", "-q"], stderr=subprocess.DEVNULL)

# Clone SAM3 INTO backend/sam3
print("\n📦 Cloning SAM3 into backend/sam3...")
BACKEND_DIR = f"{BASE_DIR}/vtrack-ai-studio/backend"
os.chdir(BACKEND_DIR)
subprocess.run(["rm", "-rf", "./sam3"])
subprocess.run(["git", "clone", "-q", "https://github.com/facebookresearch/sam3", "./sam3"])
subprocess.run([sys.executable, "-m", "pip", "install", "-e", "./sam3", "-q"], stderr=subprocess.DEVNULL)
print("✅ SAM3 installed")

# HuggingFace + checkpoint
print("\n🔐 Downloading SAM3 checkpoint...")
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login, hf_hub_download

user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
login(token=HF_TOKEN, add_to_git_credential=False)

checkpoint_dir = f"{BACKEND_DIR}/checkpoints/sam3"
checkpoints_subdir = f"{checkpoint_dir}/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(checkpoints_subdir, exist_ok=True)

checkpoint_path = hf_hub_download(
    repo_id="facebook/sam3",
    filename="sam3.pt",
    local_dir=checkpoint_dir,
    token=HF_TOKEN,
)
print(f"✅ Checkpoint: {checkpoint_path}")

# Symlinks
sam3_pt = f"{checkpoint_dir}/sam3.pt"
for dst in [f"{checkpoint_dir}/sam3_hiera_large.pt", f"{checkpoints_subdir}/sam3_hiera_tiny.pt"]:
    try:
        if os.path.islink(dst) or os.path.exists(dst): os.remove(dst)
        os.symlink(sam3_pt, dst)
    except: pass

os.chdir(BASE_DIR)
print("\n" + "="*60)
print("✅ SETUP COMPLETE! Run Cell 2 to launch.")
print("="*60)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 2: ONE-COMMAND LAUNCH (Stable, No Tunnels)
# ═══════════════════════════════════════════════════════════════════════════════
import os
import subprocess
import time
import requests

BASE_DIR = "/kaggle/working"
NODE_BIN = f"{BASE_DIR}/node_env/bin"
os.environ["PATH"] = f"{NODE_BIN}:{os.environ['PATH']}"
os.chdir(f"{BASE_DIR}/vtrack-ai-studio")

# Kill existing
subprocess.run(["pkill", "-f", "uvicorn"], stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-f", "vite"], stderr=subprocess.DEVNULL)
time.sleep(2)

# Start Backend
print("🚀 Starting Backend...")
backend_log = open(f"{BASE_DIR}/backend.log", "w")
subprocess.Popen(
    ["uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=backend_log, stderr=subprocess.STDOUT,
    cwd=f"{BASE_DIR}/vtrack-ai-studio/backend"
)

# Wait for backend
for i in range(20):
    try:
        r = requests.get("http://localhost:8000/api/health", timeout=3)
        print(f"✅ Backend ready - SAM3: {r.json().get('sam3_available')}")
        break
    except:
        print(f"⏳ Waiting... ({i+1}/20)")
        time.sleep(2)

# Start Frontend
print("\n🚀 Starting Frontend...")
with open(".env", "w") as f:
    f.write("VITE_API_URL=http://localhost:8000\n")

subprocess.run([f"{NODE_BIN}/npm", "install", "--silent"], stderr=subprocess.DEVNULL)

frontend_log = open(f"{BASE_DIR}/frontend.log", "w")
subprocess.Popen(
    [f"{NODE_BIN}/npm", "run", "dev", "--", "--host", "0.0.0.0", "--port", "4173"],
    stdout=frontend_log, stderr=subprocess.STDOUT,
    cwd=f"{BASE_DIR}/vtrack-ai-studio"
)
time.sleep(10)

print("\n" + "="*60)
print("🎉 VTrackAI Studio LIVE!")
print("="*60)
print("\n📱 Frontend: http://localhost:4173")
print("🔧 Backend:  http://localhost:8000/api/health")
print("\n⚠️ Kaggle: Click 'External link' button above to access UI!")
print("   (No tunnel passwords required)")
print("="*60)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 3: HEALTH & LOGS (Run to check status)
# ═══════════════════════════════════════════════════════════════════════════════
import requests
import subprocess

BASE_DIR = "/kaggle/working"

print("=" * 60)
print("=== Health Check ===")
print("=" * 60)

try:
    r = requests.get("http://localhost:8000/api/health", timeout=5)
    health = r.json()
    print(f"✅ Status: {health.get('status')}")
    print(f"   SAM3 Available: {health.get('sam3_available')}")
    print(f"   SAM3 Initialized: {health.get('sam3_initialized')}")
    print(f"   Device: {health.get('device')}")
except Exception as e:
    print(f"❌ Backend not responding: {e}")

print("\n=== Backend Log (last 15 lines) ===")
try:
    with open(f"{BASE_DIR}/backend.log") as f:
        lines = f.readlines()[-15:]
        print("".join(lines))
except:
    print("No log yet")

print("\n=== Frontend Log (last 10 lines) ===")
try:
    with open(f"{BASE_DIR}/frontend.log") as f:
        lines = f.readlines()[-10:]
        print("".join(lines))
except:
    print("No log yet")

print("\n" + "=" * 60)
print("✅ Copy External link from Kaggle → Open VTrackAI Studio")
print("=" * 60)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 4: CLEANUP
# ═══════════════════════════════════════════════════════════════════════════════
import subprocess
print("🧹 Cleaning up...")
subprocess.run(["pkill", "-f", "uvicorn"], stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-f", "node"], stderr=subprocess.DEVNULL)
print("✅ Done")